## Wrapper function to load augmented dataset & run a CNN or HQNN setup from HybridNN

build_and_run_nn:
- Classical: pass None for qubit_count and quantum_layer_ars
- Quantum layer: include the above information to incorporate one quantum layer in HybridNN

The base classical CNN network was set up to be simple enough to allow the incorporation of a Quantum Layer to significantly impact the model's capability to learn patterns in the dataset. 


See hybrid_nn.py for the setup. Core features are:
- 2 CNN layers with batch normalization followed by pooling. 
- 2 fully connected linear layers for the classical route
- linear layer -> quantum layer -> linear layer for the hybrid-quantum route


The dataset is not significantly large (~1500 samples for the 5 leaf labels chosen in config.yml). 
- dropout, batch normalization, & limiting the number of linear neurons help significantly to reduce overfitting.


See model_setup.py for the training algorithm. Core features are:
- batch processing followed by accuracy metrics per epoch on the entire train and test data
- learning rate scheduling (starts high and reduces by half every step_size=10 epochs)
- early stopping from trainig plateaus or nearing overfitting when training accuracy reaches above 99.5%

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import torch
from cudaq import spin
from src.model_builder import build_and_run_nn

quantum_layer = {
    'hamiltonians': [spin.z(0) * spin.x(0), 
                     spin.z(1) * spin.x(1), 
                     spin.z(2) * spin.x(2), 
                     spin.z(3) * spin.x(3), 
                     spin.z(4) * spin.x(4)],

    'shift': 10*torch.tensor(torch.pi / 4),
    'quantum_param_init_scale': 0.1,
    'features_per_qubit': 2,
    'n_classes': 5}

# Quantum ex.
# result = build_and_run_nn('categories', 
#                           conv_channels_1=32, conv_channels_2=64, 
#                           fc_neurons_1=64, fc_neurons_2=32, 
#                           qubit_count=5, quantum_layer_args=quantum_layer)

# Classical ex.
result = build_and_run_nn('categories', 
                          conv_channels_1=32, conv_channels_2=64, 
                          fc_neurons_1=64, fc_neurons_2=32, 
                          qubit_count=None, quantum_layer_args=None)

file_name = result[-1]
print(result[-2])



df = pd.read_csv(file_name + '.csv')

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(df['training_cost'], label='Train')
plt.plot(df['testing_cost'], label='Test')
plt.xlabel('Epochs')
plt.ylabel('Cost')
plt.legend()
plt.xlim(0,)
plt.ylim(0,)

plt.subplot(1, 2, 2)
plt.plot(df['training_accuracy'], label='Train')
plt.plot(df['testing_accuracy'], label='Test')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.xlim(0,)
plt.ylim(0,1.01)

plt.tight_layout()
plt.show()